Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [12]:
batch_size = 16
patch_size = 5
depth = 16 # depth of the neural network/filter
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    # input, filter, strides, paddding
    # stride of the sliding window for each dimension of input
    # For the most common case of the same horizontal and vertices strides, strides = [1, stride, stride, 1]
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [13]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.511044
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 200: 0.622460
Minibatch accuracy: 68.8%
Validation accuracy: 78.0%
Minibatch loss at step 400: 0.180896
Minibatch accuracy: 93.8%
Validation accuracy: 78.5%
Minibatch loss at step 600: 0.334375
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Minibatch loss at step 800: 0.481611
Minibatch accuracy: 87.5%
Validation accuracy: 82.1%
Minibatch loss at step 1000: 0.784536
Minibatch accuracy: 75.0%
Validation accuracy: 83.4%
Test accuracy: 89.7%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

#### Format
tf.nn.max_pool(value, ksize, strides, padding, data_format='NHWC', name=None)

Args:

__value__: A 4-D Tensor with shape [batch, height, width, channels] and type tf.float32.

**ksize**: A list of ints that has length >= 4. The size of the window for each dimension of the input tensor.

**strides**: A list of ints that has length >= 4. The stride of the sliding window for each dimension of the input tensor.

**padding**: A string, either 'VALID' or 'SAME'. The padding algorithm. See the comment here

**data_format**: A string. 'NHWC' and 'NCHW' are supported.

**name**: Optional name for the operation.


In [14]:
batch_size = 16
patch_size = 5
depth = 16 # depth of the neural network/filter
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    # input, filter, strides, paddding
    # stride of the sliding window for each dimension of input
    # For the most common case of the same horizontal and vertices strides, strides = [1, stride, stride, 1]
    conv_pool = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], padding = 'SAME')
    hidden = tf.nn.relu(conv_pool + layer1_biases)
    
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    conv_pool = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], padding = 'SAME')
    hidden = tf.nn.relu(conv_pool + layer2_biases)
    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [15]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.037072
Minibatch accuracy: 12.5%
Validation accuracy: 8.0%
Minibatch loss at step 200: 0.754317
Minibatch accuracy: 62.5%
Validation accuracy: 77.7%
Minibatch loss at step 400: 0.160227
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 600: 0.250880
Minibatch accuracy: 93.8%
Validation accuracy: 82.2%
Minibatch loss at step 800: 0.457448
Minibatch accuracy: 87.5%
Validation accuracy: 82.7%
Minibatch loss at step 1000: 0.641135
Minibatch accuracy: 75.0%
Validation accuracy: 84.2%
Test accuracy: 90.0%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

### OPTION 1: Dropout

In [25]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    keep_p = tf.placeholder(tf.float32)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        conv_pool = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], padding = 'SAME')
        hidden = tf.nn.relu(conv_pool + layer1_biases)
        hidden = tf.nn.dropout(hidden, keep_prob = keep_p)
    
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        conv_pool = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], padding = 'SAME')
        hidden = tf.nn.relu(conv_pool + layer2_biases)
        hidden = tf.nn.dropout(hidden, keep_prob = keep_p)
    
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [24]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_p: 0.75}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict = feed_dict)
    
    if (step % 200 == 0):
        print('Minibatch loss at step %d: %f' % (step, l))
        print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
        
        feed_dict = {keep_p: 1}
        print('Validation accuracy: %.1f%%' % accuracy(
        session.run(valid_prediction, feed_dict = feed_dict), valid_labels))
        
        print('Test accuracy: %.1f%%' % accuracy(
                session.run(test_prediction, feed_dict = feed_dict), test_labels))

Initialized
Minibatch loss at step 0: 4.347401
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Test accuracy: 10.0%
Minibatch loss at step 200: 1.175437
Minibatch accuracy: 56.2%
Validation accuracy: 72.2%
Test accuracy: 78.7%
Minibatch loss at step 400: 0.325623
Minibatch accuracy: 93.8%
Validation accuracy: 77.2%
Test accuracy: 83.8%
Minibatch loss at step 600: 0.495382
Minibatch accuracy: 87.5%
Validation accuracy: 81.9%
Test accuracy: 88.5%
Minibatch loss at step 800: 0.659331
Minibatch accuracy: 75.0%
Validation accuracy: 82.6%
Test accuracy: 89.4%
Minibatch loss at step 1000: 1.076657
Minibatch accuracy: 62.5%
Validation accuracy: 83.0%
Test accuracy: 89.3%


### OPTION 2: Learning rate decay

In [14]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    keep_p = tf.placeholder(tf.float32)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        conv_pool = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], padding = 'SAME')
        hidden = tf.nn.relu(conv_pool + layer1_biases)
        hidden = tf.nn.dropout(hidden, keep_prob = keep_p)
    
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        conv_pool = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], padding = 'SAME')
        hidden = tf.nn.relu(conv_pool + layer2_biases)
        hidden = tf.nn.dropout(hidden, keep_prob = keep_p)
    
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    global_step = tf.Variable(0, trainable = False)  # count the number of steps taken.
    starter_learning_rate = 0.05
    learning_rate = tf.train.exponential_decay(starter_learning_rate, 
                                               global_step = global_step,
                                               decay_steps = 10000,
                                              decay_rate = 0.5,
                                              staircase = True)
    
    optimizer = tf.train.GradientDescentOptimizer(
        learning_rate).minimize(loss,global_step = global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [15]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_p: 0.75}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict = feed_dict)
    
    if (step % 200 == 0):
        print('Minibatch loss at step %d: %f' % (step, l))
        print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
        
        feed_dict = {keep_p: 1}
        print('Validation accuracy: %.1f%%' % accuracy(
        session.run(valid_prediction, feed_dict = feed_dict), valid_labels))
        
        print('Test accuracy: %.1f%%' % accuracy(
                session.run(test_prediction, feed_dict = feed_dict), test_labels))

Initialized
Minibatch loss at step 0: 4.755794
Minibatch accuracy: 6.2%
Validation accuracy: 10.6%
Test accuracy: 10.7%
Minibatch loss at step 200: 1.031684
Minibatch accuracy: 56.2%
Validation accuracy: 65.1%
Test accuracy: 71.4%
Minibatch loss at step 400: 0.318033
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Test accuracy: 86.0%
Minibatch loss at step 600: 0.602139
Minibatch accuracy: 75.0%
Validation accuracy: 82.3%
Test accuracy: 88.7%
Minibatch loss at step 800: 0.535681
Minibatch accuracy: 75.0%
Validation accuracy: 83.5%
Test accuracy: 90.0%
Minibatch loss at step 1000: 0.820686
Minibatch accuracy: 75.0%
Validation accuracy: 84.2%
Test accuracy: 90.4%
Minibatch loss at step 1200: 0.294428
Minibatch accuracy: 93.8%
Validation accuracy: 84.7%
Test accuracy: 90.7%
Minibatch loss at step 1400: 0.372117
Minibatch accuracy: 87.5%
Validation accuracy: 85.3%
Test accuracy: 91.0%
Minibatch loss at step 1600: 0.674830
Minibatch accuracy: 87.5%
Validation accuracy: 85.9%
Test acc

### OPTION 3: Dropout and learning rate decay

In [10]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    keep_p = tf.placeholder(tf.float32)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        conv_pool = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], padding = 'SAME')
        hidden = tf.nn.relu(conv_pool + layer1_biases)
        hidden = tf.nn.dropout(hidden, keep_prob = keep_p)
    
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        conv_pool = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], padding = 'SAME')
        hidden = tf.nn.relu(conv_pool + layer2_biases)
        hidden = tf.nn.dropout(hidden, keep_prob = keep_p)
    
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    global_step = tf.Variable(0, trainable = False)  # count the number of steps taken.
    starter_learning_rate = 0.05
    learning_rate = tf.train.exponential_decay(starter_learning_rate, 
                                               global_step = global_step,
                                               decay_steps = 10000,
                                              decay_rate = 0.5,
                                              staircase = True)
    
    optimizer = tf.train.GradientDescentOptimizer(
        learning_rate).minimize(loss,global_step = global_step)
    
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_p: 0.75}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict = feed_dict)
    
    if (step % 200 == 0):
        print('Minibatch loss at step %d: %f' % (step, l))
        print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
        
        feed_dict = {keep_p: 1}
        print('Validation accuracy: %.1f%%' % accuracy(
        session.run(valid_prediction, feed_dict = feed_dict), valid_labels))
        
        print('Test accuracy: %.1f%%' % accuracy(
                session.run(test_prediction, feed_dict = feed_dict), test_labels))

Initialized
Minibatch loss at step 0: 2.836685
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Test accuracy: 10.0%
Minibatch loss at step 200: 0.939896
Minibatch accuracy: 68.8%
Validation accuracy: 69.8%
Test accuracy: 75.4%
Minibatch loss at step 400: 0.316336
Minibatch accuracy: 87.5%
Validation accuracy: 80.1%
Test accuracy: 86.2%
Minibatch loss at step 600: 0.438243
Minibatch accuracy: 87.5%
Validation accuracy: 82.4%
Test accuracy: 88.7%
Minibatch loss at step 800: 0.717850
Minibatch accuracy: 75.0%
Validation accuracy: 83.6%
Test accuracy: 89.6%
Minibatch loss at step 1000: 0.782204
Minibatch accuracy: 68.8%
Validation accuracy: 84.7%
Test accuracy: 90.3%
